# `TohuNamespaceNEW2`


[TOC]


The class `TohuNamespaceNEW2` allows grouping together other tohu generators and treating them as a single unit (which is used to implement the functionality of custom generators).

## Initialisation and adding field generators

In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.tohu_namespace_NEW_2 import TohuNamespaceNEW2

In [2]:
g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

Note that when a field generator is added to the tohu namesapce, internally a new spawn is created. We can verify this by checking that the generators are different, for example in the case of `g1` and the field generator `aa`:

In [3]:
print(f"g1: {g1}")
print(f"aa: {tohu_namespace.field_generators['aa']}")

assert tohu_namespace.field_generators['aa'] is not g1
assert not tohu_namespace.field_generators['aa'].is_clone_of(g1)

g1: <Integer (id=529fe7)>
aa: <Integer (id=789a65)>


There is a convenience method `add_field_generators_from_dict()` which allows passing a dictionary, and it will call `add_field_generator()` for any tohu generators found in this dictionary (while ignoring any other values).

In [4]:
dct = {
    "aa": Integer(100, 200),
    "bb": HashDigest(length=6),
    "some_string": "this string will not be added because it is not a tohu generator",
    "cc": FakerGenerator(method="name"),
    "answer": 42  # this number will also not be added because it is not a tohu generator
}

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_field_generators_from_dict(dct)

In [5]:
tohu_namespace.field_generators

{'aa': <Integer (id=321e09)>,
 'bb': <HashDigest (id=d52f17)>,
 'cc': <FakerGenerator (id=c5aee9)>}

## Adding non-field generators

In [6]:
from tohu.looping_NEW import LoopVariableNEW

In [7]:
xx = LoopVariableNEW("xx", values=[4, 5, 6])

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)

In [8]:
tohu_namespace.field_generators

{}

In [9]:
tohu_namespace.all_generators

{'xx': <LoopVariable: name='xx', loop_level=None, values=[4, 5, 6], cur_value=4 (tohu_id=004f59)>}

Note that in this case the generator which is internally stored in the tohu namespace is actually a clone of `xx`.

In [10]:
assert tohu_namespace.all_generators["xx"].is_clone_of(xx)

## Setting the `tohu_items_cls` attribute

Let's create a new tohu namespace and add both field generators and non-field generators.

In [11]:
xx = LoopVariableNEW("xx", values=[111, 222]).set_loop_level(2)
yy = LoopVariableNEW("yy", values=["AAA", "BBB"]).set_loop_level(1)

g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)
tohu_namespace.add_non_field_generator("yy", yy, is_externally_managed=True)
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

Initially the `tohu_items_cls` attribute refers to a non-existent tohu items class:

In [12]:
tohu_namespace.tohu_items_cls

<NonExistentTohuItemsClass>

Once all desired generators have been added to the tohu namespace, we can call `set_tohu_items_class`, which will automatically create a tohu items class with the same field names as the generators contained in the namespace.

In [13]:
tohu_namespace.set_tohu_items_class(name="Quux")

In [14]:
tohu_namespace.tohu_items_cls

tohu.tohu_items_class.Quux

This items class can then be used to create individual tohu items.

In [15]:
tohu_namespace.tohu_items_cls(aa=100, bb="910A97", cc="Kristen Wallace")

Quux(aa=100, bb='910A97', cc='Kristen Wallace')

## Resetting and generating tohu items

In [16]:
tohu_namespace.all_generators

{'xx': <LoopVariable: name='xx', loop_level=2, values=[111, 222], cur_value=111 (tohu_id=1d2fc7)>,
 'yy': <LoopVariable: name='yy', loop_level=1, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=332ee9)>,
 'aa': <Integer (id=261762)>,
 'bb': <HashDigest (id=92d610)>,
 'cc': <FakerGenerator (id=69fe0b)>}

In [17]:
tohu_namespace.field_generators

{'aa': <Integer (id=261762)>,
 'bb': <HashDigest (id=92d610)>,
 'cc': <FakerGenerator (id=69fe0b)>}

In [18]:
tohu_namespace.reset(seed=11111)

print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


## Adding generators with dependencies

In [19]:
from tohu.derived_generators import Apply

In [20]:
aa = Integer(1, 9)
bb = Apply(lambda x: x*11, aa)
cc = Apply(lambda x: x*101, bb)

In [21]:
assert bb.arg_gens[0].is_clone_of(aa)
assert cc.arg_gens[0].is_clone_of(bb)

In [22]:
tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_field_generator("rr", aa)
tohu_namespace.add_field_generator("ss", aa)
tohu_namespace.add_field_generator("tt", bb)
tohu_namespace.add_field_generator("uu", cc)
tohu_namespace.add_field_generator("vv", cc)
tohu_namespace.set_tohu_items_class("Quux")

In [23]:
tohu_namespace.field_generators

{'rr': <Integer (id=08c8d2)>,
 'ss': <Integer (id=e6b940)>,
 'tt': <Apply (id=04f956)>,
 'uu': <Apply (id=2915d7)>,
 'vv': <Apply (id=2c266f)>}

Note that even though the generator `aa` is added to the namespace twice (first with the name `"rr"` and then with the name `"ss"`), `ss` actually ends up as a _clone_ of `rr` (this is to ensure that the tohu items produced by the namespace contain the correct values).

The following checks that this works as expected.

In [24]:
assert tohu_namespace.field_generators["ss"].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["tt"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["rr"])
assert tohu_namespace.field_generators["uu"].arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])
assert tohu_namespace.field_generators["vv"].parent.arg_gens[0].is_clone_of(tohu_namespace.field_generators["tt"])

tohu_namespace.reset(seed=11111)
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))
print(next(tohu_namespace))

Quux(rr=8, ss=8, tt=88, uu=8888, vv=8888)
Quux(rr=9, ss=9, tt=99, uu=9999, vv=9999)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=6, ss=6, tt=66, uu=6666, vv=6666)
Quux(rr=3, ss=3, tt=33, uu=3333, vv=3333)


## Extracting loop variables from a tohu namespace into a loop runner

The method `extract_loop_runner()` finds all loop variables in the tohu namespace and creates a `LoopRunner` instance containing them. This is used internally by custom generators to orchestrate looping.

In [25]:
xx = LoopVariableNEW("xx", values=[111, 222]).set_loop_level(2)
yy = LoopVariableNEW("yy", values=["AAA", "BBB"]).set_loop_level(1)

g1 = Integer(100, 200)
g2 = HashDigest(length=6)
g3 = FakerGenerator(method="name")

tohu_namespace = TohuNamespaceNEW2()
tohu_namespace.add_non_field_generator("xx", xx, is_externally_managed=True)
tohu_namespace.add_non_field_generator("yy", yy, is_externally_managed=True)
tohu_namespace.add_field_generator("aa", g1)
tohu_namespace.add_field_generator("bb", g2)
tohu_namespace.add_field_generator("cc", g3)

In [26]:
loop_runner = tohu_namespace.extract_loop_runner()
loop_runner.loop_variables

{'xx': <LoopVariable: name='xx', loop_level=2, values=[111, 222], cur_value=111 (tohu_id=529fe7)>,
 'yy': <LoopVariable: name='yy', loop_level=1, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=5d0a1c)>}

Note that the loop variables in the loop runner are the exact same Python objects as the ones present in the tohu namespace. This allows a custom generator to orchestrate looping via the loop runner and ensure that field generators in the tohu namespace which depend on the loop variables produce the correct, up-to-date values.

In [27]:
assert loop_runner.loop_variables["xx"] is tohu_namespace.all_generators["xx"]
assert loop_runner.loop_variables["xx"] is tohu_namespace.all_generators["xx"]

**TODO:**

- [ ] Add test that field generators are reset when the tohu namespace is reset, but externally managed (non-field?) generators are left alone.